## Imports

In [45]:
import tkinter
import customtkinter
import cv2
from PIL import Image, ImageTk
import numpy as np
from keras.models import model_from_json

## Load Model

In [46]:
jsonFile = open('models/emotionModel.json', 'r')
loadedModel = jsonFile.read()
jsonFile.close()
emotionModel = model_from_json(loadedModel)

In [47]:
emotionModel.load_weights('models/savedModel.h5')

### Emotions

In [48]:
emotionDict = {
    0: 'Angry',
    1: 'Disgusted',
    2: 'Fearful',
    3: 'Happy',
    4: 'Neutral',
    5: 'Sad',
    6: 'Surprised'}

## Create App

In [49]:
root = customtkinter.CTk()
customtkinter.set_appearance_mode("Dark")
customtkinter.set_default_color_theme("green")

In [50]:
root.geometry("940x640")

In [51]:
frame = customtkinter.CTkFrame(master = root)
frame.pack(pady = 60, padx = 60, fill = "both", expand = True)

In [52]:
title = customtkinter.CTkLabel(master = frame, text = "Face Expression Detector", font = ("Roboto", 24))
title.pack(pady = 12, padx = 10)

In [53]:
cascPath = os.path.dirname(cv2.__file__)+"/data/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

## Set Up Camera

In [54]:
camera = cv2.VideoCapture(0)

## Set Detection and Display

In [55]:
while True:
    ret, frames = camera.read()
    gray = cv2.cvtColor(frames, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frames, (x, y), (x+w, y+h), (60, 255, 60), 2)
        grayFrame = gray[y:y + h, x:x + w]
        croppedImage = np.expand_dims(np.expand_dims(cv2.resize(grayFrame, (48, 48)), -1), 0)
        
        prediction = emotionModel.predict(croppedImage)
        maxIndex = int(np.argmax(prediction))
        
        cv2.putText(frames, emotionDict[maxIndex], (x, y-20), cv2.FONT_HERSHEY_DUPLEX, 1, (60, 255, 60), 1, cv2.LINE_AA)
        
    img = frames
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = customtkinter.CTkImage(Image.fromarray(img), size = (720, 520))
    button = customtkinter.CTkButton(master = frame, image = img, width = 720, height = 520, text='', corner_radius = 0)
    button.place(relx = 0.5, rely = 0.5, anchor = tkinter.CENTER)
    
    root.update()

1/1 [==============================] - 0s 39ms/step


TclError: can't invoke "frame" command: application has been destroyed